In [59]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
import json
from scipy.special import lpmn  # associated legendre polynomial in sequence

# sys.path.append(os.getcwd())
# sys.path.append(os.path.join(os.getcwd(), ".."))
# sys.path.append(os.path.join(os.getcwd(), "..", "gtracr"))

from gtracr.trajectory import Trajectory

In [119]:
# where the data is stored
data_path = os.path.join(os.getcwd(), "..", "..", "data")
print(os.listdir(data_path))
# we want to store the data as a .json format
# so lets convert the original data, saved as a .txt file
# and save it as a .json file now.
N = 13   #the degree of truncation

yearly_coeffdict = {
    "g": {
        "values":np.zeros((N+1,N+1)).tolist(), 
        "svp":np.zeros((N+1, N+1)).tolist()
    },          
    "h": {
        "values":np.zeros((N+1,N+1)).tolist(), 
        "svp":np.zeros((N+1, N+1)).tolist()
    }  
}

with open(os.path.join(data_path, "IGRF13.txt"), "r") as f:
    for i, line in enumerate(f.readlines()):
        # skip if we see "#" symbol (indicates comments)
        # or any line before dates (extra information thats not needed)
        if line[0] == "#" or i < 4:  # a comment
            continue
        # get the years of the magnetic field coefficient data as keys to our dictionary
        # here we also construct 2 separate arrays for the 'g' and 'h' coefficients
        # presented as a 13x14 data structure (14 to account for m=0 case)
        # the matrix will be sparse, but would allow for easier calculations this way
        elif i == 4:
            years = np.fromstring(line, sep=' ')
#             print(years)
            igrfcoeff_dict = {int(year) : yearly_coeffdict for year in years}
        
        # else we store the coeffieicents in array of shape ((N=13, N=13))
        # for corresponding n and m values for the coefficients
        elif i > 4:
            line_data = np.fromstring(line, sep=' ')
            (n,m) = line_data[:2].astype(int)
#             print(n,m)
            coeff_data = line_data[2:]
    
            row_index = n  # we consider n=0 case as well for Legendre polynomial to work
        
            if m < 0:   # m < 0 since we indicate h coefficeint with m < 0
                coeff_var = "h"
                column_index = -m
            else:
                coeff_var = "g"
                column_index = m
            for j, coeff in enumerate(coeff_data):
                # append coeffients to year array in coefficient dictionary
                list(igrfcoeff_dict.values())[j][coeff_var]["values"][row_index][column_index] = coeff
                
                # evaluate the secular variation parameter (svp)
                # (gnm(T0+5) - gnm(T0-5)) / 2*5 from midpoint method
                if j > 0 and j < len(coeff_data)-1:
                    svp_mn = (coeff_data[j+1] - coeff_data[j-1]) / 10.
                    list(igrfcoeff_dict.values())[j][coeff_var]["svp"][row_index][column_index] = svp_mn

['IGRF13.txt', 'igrf13_coeffs.json']


In [120]:
# dump to json file so that we can use this again

with open(os.path.join(data_path, "igrf13_coeffs.json"), "w") as f:
    json.dump(igrfcoeff_dict, f, indent=2)

In [121]:
# open the json file
# this works properly :)
with open(os.path.join(data_path, "igrf13_coeffs.json"), "r") as f:
    igrfcoeff_dict = json.load(f)
print(igrfcoeff_dict.keys())

dict_keys(['1900', '1905', '1910', '1915', '1920', '1925', '1930', '1935', '1940', '1945', '1950', '1955', '1960', '1965', '1970', '1975', '1980', '1985', '1990', '1995', '2000', '2005', '2010', '2015', '2020', '2025'])


In [122]:
# modify the Gauss coefficients to get the secular variation of the coeffiencts
# they obey the form gnm(t) = gnm(T0) + gnm'(T0)*(t-T0)
# i.e. a linear relationship
# where gnm'(T0) is the secular variation
# approximated by (gnm(T0+5) - gnm(T0-5)) / 2*5 
# using midpoint method (since we have data in 5 year intervals)
# T0 is the epoch directly preceding t 
# for example if t = 2016, then T0 = 2015

# g, h array
years = np.array(list(igrfcoeff_dict.keys())).astype(int)
print(years)
# g_arr = np.array([list(igrfcoeff_dict.values())[i]["g"] for i in range(len(years))])
# h_arr = np.array([list(igrfcoeff_dict.values())[i]["h"]for i in range(len(years))])

# print(g_arr)

# the Gauss coefficient at a certain n, m
# t is in years (0 being at 0 A.D.)
# coeff_var is either "g" or "h", should complain if either
# of these are not provided
# n,m are in convention with the legendre polynomials as well as the array
# index itself
# some condition to limit t < 1905 or t > 2020 should be implemented
def gauss_coeff(t, coeff_var, n, m):
    
    # complain if g or h not provided
    if coeff_var.find("g") < 0 and coeff_var.find("h") < 0:
        raise Exception("Only correct Gauss coefficient inputs (g or h) are valid!")
        
    # complain if time is greater / less than allowed
    if t < 1905 or t > 2020:
        raise Exception("Invalid time scale, lower or raise timescale!")
    
    nearest_index = np.argmin(np.abs(years - t))
    epoch = years[nearest_index]   # T0
    
    print(epoch)
    
    epoch_coeff = igrfcoeff_dict[str(epoch)][coeff_var]["values"][n][m]  # gnm(T0)
    
#     epoch_sv = (igrfcoeff_dict[str(epoch+5)][coeff_var][n][m] - igrfcoeff_dict[str(epoch-5)][coeff_var][n][m]) / 10  # (gnm(T0+5) - gnm(T0-5)) / 2*5  
    epoch_svp = igrfcoeff_dict[str(epoch)][coeff_var]["svp"][n][m]  # gmn'(t), secular variation parameter
    
    return epoch_coeff + epoch_svp * (t - epoch)


[1900 1905 1910 1915 1920 1925 1930 1935 1940 1945 1950 1955 1960 1965
 1970 1975 1980 1985 1990 1995 2000 2005 2010 2015 2020 2025]


In [124]:
print(gauss_coeff(2014, "g", 2, 2))

2015
1667.485


In [ ]:
# define global variables
RE = 6.3712 * (1e6)
N = 13   # degree of truncation

In [ ]:
# define the magnetic field components
# in spherical coordinates
# inputs are the 3-vector (r, theta, phi) + time t

# the r component
def Br(t, r, theta, phi):
    # array of associated legendre polynomials
    # ranging from 0 to 13 (inclusive) for both m and n
    # at cos(theta)
    lp_mn, _ = lpmn(N, N, np.cos(theta))  